# Graficar predicciones

Este notebook toma un archivo de predicciones (`FIC_ENTRADA` joblib), generado por **05a_ModeloRNN-DirectApproach.ipynb** o **05b_Transformers.ipynb**, y los combina con los archivos originales de datos sintéticos (`FICHERO_INICIAL_SIN_PARADA` y `FICHERO_INICIAL`). Luego, procesa los datos y genera un archivo HTML interactivo que permite visualizar los gráficos completos, incluyendo las predicciones, con opciones de zoom y exploración.

In [1]:
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pickle
import plotly.graph_objects as go
import plotly
import plotly.io as pio
import dash
from dash import dcc, html
import webbrowser

# Variables globales

In [2]:
FIC_ENTRADA='prediccion_rnn_direct_LSTM4.joblib'
RUTA_FICHEROS = r"C:\Users\jaume\Documents\Proyecto\datos"
FICHERO_INICIAL = "02_datos_limpios.pkl"
FICHERO_INICIAL_SIN_PARADA = "02_datos_limpios_sin_parada.pkl"

# Parametros
WINDOW_SIZE=30
OFFSET=4

# Cargamos ficheros

Cargo todos los ficheros para poder representar las predicciones desescaladas.

In [3]:
# Carga de datos
os.chdir(RUTA_FICHEROS)

with open(FIC_ENTRADA, 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
#df_results = data_dict['df_results'].copy()
predictions_test = data_dict['predictions_test'].copy()


# Cargar desde el archivo
with open('train_test_val_scaler.joblib', 'rb') as file:
    data_dict = joblib.load(file)

# Recuperar DataFrame y scaler
df_train = data_dict['train'].copy() 
df_test = data_dict['test'].copy() 
df_val = data_dict['val'].copy() 
scaler = data_dict['scaler']              # Es el StandardScaler que entrenaste en [L,A,B,durac_log,tiempo_log,vel]

# Codigo para desensamblar

Como guardamos más datos normalizados que no solo LAB, aquí lo que se hace es separarlos para luego usarlos.

In [4]:
# Crear un nuevo scaler solo con los primeros 3 componentes (L, A, B)
scaler_3d = StandardScaler()
scaler_3d.mean_ = scaler.mean_[:3]  # Primeras 3 medias (L, A, B)
scaler_3d.scale_ = scaler.scale_[:3]  # Primeras 3 desviaciones (L, A, B)

# Aplicar inverse_transform al DataFrame Y_pred_rnn
Y_pred_desescalado = scaler_3d.inverse_transform(predictions_test)

# Mostrar resultados
print(Y_pred_desescalado[:10])

[[ 61.634785    6.2744765 -18.867989 ]
 [ 61.658554    6.1794715 -18.912199 ]
 [ 61.6543      6.0847545 -18.935308 ]
 [ 61.68084     6.0000834 -19.01892  ]
 [ 61.65538     5.9065776 -19.0247   ]
 [ 61.675934    5.7999244 -19.07196  ]
 [ 61.61671     5.633323  -18.98359  ]
 [ 61.52899     5.432361  -18.700514 ]
 [ 61.53232     5.2043395 -18.634102 ]
 [ 61.491425    4.8911204 -18.552084 ]]


Agrego a las predicciones los resultados ya desescalados en la fila del dataframe que corresponde.

In [5]:
offset_inicial = WINDOW_SIZE + OFFSET - 1
offset_final = offset_inicial + len(Y_pred_desescalado)

df_test_pred = df_test.iloc[offset_inicial:offset_final].copy()
df_test_pred['L_pred'] = Y_pred_desescalado[:,0]
df_test_pred['A_pred'] = Y_pred_desescalado[:,1]
df_test_pred['B_pred'] = Y_pred_desescalado[:,2]

Estos ficheros que cargo aquí son los originales, a los que luego les añado las predicciones en la posición adecuada ahora juntadas por el índice que ya es el datatime es decir, el momento donde se ha tomado la muestra original

In [6]:
# Este lo usaré al final para ensamblar
with open(FICHERO_INICIAL, 'rb') as file:
    df_inicial = pickle.load(file)

# Este lo usaré al final para ensamblar
with open(FICHERO_INICIAL_SIN_PARADA, 'rb') as file:
    df_inicial_sin_parada = pickle.load(file)

In [7]:
df_merged = df_inicial_sin_parada.merge(
    df_test_pred[['L_pred', 'A_pred', 'B_pred']], 
    left_on='seq_time',  # Usa la columna en df_test
    right_index=True,    # Usa el índice en df_test_pred
    how='left'           # Mantiene todas las filas de df_test, rellenando NaN si no hay coincidencias
)


In [8]:
# Seleccionar las columnas de interés de df_filtered
columns_to_add = ["L_pred", "A_pred", "B_pred"]

# Combinar los DataFrames basados en el índice
df_final = df_inicial.join(df_merged[columns_to_add], how="left")

# Gráfico final predicciones

Como genero gráficos pesadas ya que son interactivos, haré que se muestren en una pestaña nueva.

In [9]:
# Filtrar solo las columnas necesarias para las gráficas
columns_to_plot = ['velocidad', 'L', 'L_pred', 'A', 'A_pred', 'B', 'B_pred']
df_filtered = df_final[columns_to_plot]

# Asegurar que el índice es datetime y ordenado
df_filtered = df_filtered.sort_index()

# Crear la figura para velocidad
fig_velocity = go.Figure()
fig_velocity.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['velocidad'],
    mode='lines',
    name='Velocidad',
    line=dict(color='blue', width=2)
))
fig_velocity.update_layout(
    title="Velocidad a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="Velocidad",
    template="plotly_white"
)

# Crear la figura para L y L_pred
fig_L = go.Figure()
fig_L.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['L'],
    mode='lines',
    name='L (Real)',
    line=dict(color='green', width=2)
))
fig_L.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['L_pred'],
    mode='lines',
    name='L (Predicción)',
    line=dict(color='red', dash='dash', width=2)
))
fig_L.update_layout(
    title="L y L_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="L",
    template="plotly_white"
)

# Crear la figura para A y A_pred
fig_A = go.Figure()
fig_A.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['A'],
    mode='lines',
    name='A (Real)',
    line=dict(color='purple', width=2)
))
fig_A.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['A_pred'],
    mode='lines',
    name='A (Predicción)',
    line=dict(color='orange', dash='dash', width=2)
))
fig_A.update_layout(
    title="A y A_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="A",
    template="plotly_white"
)

# Crear la figura para B y B_pred
fig_B = go.Figure()
fig_B.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['B'],
    mode='lines',
    name='B (Real)',
    line=dict(color='blue', width=2)
))
fig_B.add_trace(go.Scatter(
    x=df_filtered.index,
    y=df_filtered['B_pred'],
    mode='lines',
    name='B (Predicción)',
    line=dict(color='red', dash='dash', width=2)
))
fig_B.update_layout(
    title="B y B_pred a lo largo del tiempo",
    xaxis_title="Tiempo",
    yaxis_title="B",
    template="plotly_white"
)

# Crear una página HTML que contenga todas las gráficas
html_content = f"""
<html>
<head>
    <title>Visualización de Datos LSTM</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1 style="text-align:center;">Visualización de Datos de Predicción LSTM</h1>

    <h3>Velocidad</h3>
    <div id="fig_velocity"></div>
    <script>
        var fig_velocity = {fig_velocity.to_json()};
        Plotly.newPlot('fig_velocity', fig_velocity.data, fig_velocity.layout);
    </script>

    <h3>L y L_pred</h3>
    <div id="fig_L"></div>
    <script>
        var fig_L = {fig_L.to_json()};
        Plotly.newPlot('fig_L', fig_L.data, fig_L.layout);
    </script>

    <h3>A y A_pred</h3>
    <div id="fig_A"></div>
    <script>
        var fig_A = {fig_A.to_json()};
        Plotly.newPlot('fig_A', fig_A.data, fig_A.layout);
    </script>

    <h3>B y B_pred</h3>
    <div id="fig_B"></div>
    <script>
        var fig_B = {fig_B.to_json()};
        Plotly.newPlot('fig_B', fig_B.data, fig_B.layout);
    </script>

</body>
</html>
"""

# Guardar el HTML temporalmente
html_file = "visualizacion.html"
with open(html_file, "w") as f:
    f.write(html_content)

# Abrir el archivo en una nueva pestaña del navegador
webbrowser.open("file://" + os.path.abspath(html_file))


True